In [20]:
!pip install replicate
!pip install langchain_community

In [4]:
import os
import pandas as pd
import numpy as np
from getpass import getpass
from langchain_community.llms import Replicate
from langchain_core.prompts import PromptTemplate

In [22]:
REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

··········


In [23]:
llm= Replicate(
    model="meta/meta-llama-3-70b-instruct"
)

In [24]:
User_Data=pd.DataFrame(columns=["Name","Phone Number","Problem","Resolution"])

In [25]:
def get_conversational_chain(complain, chat_history):
    prompt_template = """You are a medical experiment. You have been given the patient's complain. You have to
        diagnose the patient's disease. Identify and ask crucial question to help you narrow down and diagnose the
        patient. Ask only one question based on the response. If no chat-history is provided ask initial first question.

        Previous Conversation: {previous_conversation}

        Patient Complain:
        {complain}

        Question:
        """
    previous_conversation = " ".join([entry["user"] for entry in chat_history])
    prompt = prompt_template.format(previous_conversation=previous_conversation, complain=complain)
    response = llm.predict(prompt)
    return response

In [34]:
def get_final_solution(complain, chat_history):
    prompt_template ="""
        You are an expert chat doctor. Your task is to suggest the best diagnosing to the patient based on the complain and follow up questions.
        Do not suggest the steps to refer to doctor. Mention the diagnose and common medications for the same. You make take help from the following contexts to diagnose the disease.
        Only Return the Final Response of expected disease. Do not return the intermediate results.


        Question: the input question you must answer
        Thought: you should always think about what to do
        Action: the action to take
        Action Input: the input to the action
        Observation: the result of the action
       ... (this Thought/Action/Action Input/Observation can repeat N times)
        Final Answer: the final answer to the original input question

        Begin!

        Patient Symptoms: {complain}

        Follow Up questions and answer from patient:
        {chat_history_str}
        """

    chat_history_str = ""
    for i, entry in enumerate(chat_history):
        if 'user' in entry:
            chat_history_str += f"User: {entry['user']}\n"
        elif 'User' in entry:
            chat_history_str += f"User: {entry['User']}\n"
        chat_history_str += f"Dr: {entry['Dr']}\n"
    prompt = prompt_template.format(chat_history_str=chat_history_str, complain=complain)
    response = llm.predict(prompt)
    return response

In [27]:
name = input("How are you I'm an AI Doctor, before we get started, please tell me your name ")
num = input("And your phone number ")
print(f"Welcome {name}, How can I help you ? ")
# complain = input("What's your complaint? ")
chat_history = []
for i in range(4):
    user = input()
    response = get_conversational_chain(user, chat_history)
    chat_history.append({"user": user, "Dr": response})
    print(response)

lastly = input()
chat_history.append({"user":lastly,"Dr":"ok, I'm done questioning"})

How are you I'm an Neuroverse AI Doctor, before we get started, please tell me your name yash
And your phone number 1234
Welcome yash, How can I help you ? 
fever


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




How long have you been experiencing the fever?
2 Days


Based on the patient's complaint of having a fever for 2 days, my next question would be:

"Is the fever constant or does it come and go, and have you noticed any other symptoms such as chills, sweating, or body aches?"
i am feeling cough and running nose 


With the patient's complaint of fever for 2 days and now cough and running nose, my next question would be:

How would you describe your cough? Is it dry and hacking or are you bringing up mucus or phlegm when you cough?
dry


Based on the patient's previous complaints of fever, cough, and running nose, and now feeling dry, my next question would be:

Do you have a sore throat or any difficulty swallowing?
yeah i have some sinus issue too


In [35]:
final_response = get_final_solution(chat_history[0]['user'], chat_history)
print(final_response)



Final Answer: The patient is likely suffering from Acute Viral Rhinosinusitis. Common medications for this condition include pain relievers such as acetaminophen or ibuprofen, decongestants like pseudoephedrine, and antihistamines like loratadine.


In [36]:
User_Data.loc[len(User_Data)] = [name, num, chat_history[0]['user'], final_response]

In [37]:
User_Data.head()

,Name,Phone Number,Problem,Resolution
0,yash,1234,fever,\n\nFinal Answer:\n\nBased on the patient's sy...
1,yash,1234,fever,\n\nFinal Answer: The patient is likely suffer...


In [33]:
User_Data.to_csv('output.csv', index=False)

In [38]:
def research(med):
    prompt_template ="""
        You are an medical expert , researcher and guide. Your task is to tell me completely about {med}
        """

    prompt = prompt_template.format(med=med)
    response = llm.predict(prompt)
    return response

In [ ]:
final_response = research("metoprolol")
print(final_response)